In [ ]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pathlib
from datetime import datetime
import pytz
from pytz import timezone

def crawl_content(url, category) :
  driver = webdriver.Chrome('chromedriver',options=options)  

  driver.get(url)

  # Wait for initialize, in seconds
  wait = WebDriverWait(driver, 20)

  #obtain the link for all the category tabs
  category_link = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.nav-item.d-inline-block.py-3.px-3")))
  category_link_list = []
    
  for i in category_link:
    links = i.get_property('href')
    category_link_list.append(links)
  
  #crawl the content for all the news under each category tab
  c = 0
  for c in category_link_list:  
    if(c[-5:] != "video"):

      reload = True
      while reload:  
 
        load = True
        while load:
          try:
            driver.get(c)
            time.sleep(8)
            field_category = driver.find_element_by_css_selector('span.field-category').text
            load = False 

          except:
            load = True

        category_tab = driver.execute_script("return document.querySelector('div.category-nav > a.nav-item.d-inline-block.py-3.px-3.active').innerText")

        if category_tab == 'BUSINESS TIMES':
          category_tab = 'BUSINESS'

        if field_category != category_tab:
          reload = True

        else:
          reload = False 

      #check the last news article crawled
      csv_file_checking = 'drive/My Drive/fyp/seleniumProject/checking_news.csv'
      file = pathlib.Path(csv_file)
      file2 = pathlib.Path(csv_file_checking)
      article_list = []
      count = 0
      nextPage = True
      while nextPage:    
        time.sleep(8)   
        all_news_link = driver.find_elements_by_css_selector("a.d-flex.article.listing.mb-3.pb-3")

        listOflinks=[]
        for el in all_news_link:
          link = el.get_property('href')
          listOflinks.append(link)

        time.sleep(8)        
                
        valid = True
        condition = False
        i = 0
        while (i < len(listOflinks)) and valid:
          
          driver.get(listOflinks[i])
          time.sleep(8)

          #time created of the news articles
          time_created = driver.execute_script("return document.querySelector('div.article-meta > div:last-child').innerText.split('-').pop().trim();")
          time_created = datetime.strptime(time_created, '%B %d, %Y  @ %I:%M%p')
          timezone = pytz.timezone("Asia/Kuala_Lumpur")
          time_created = time_created.astimezone(timezone)
          time_created = time_created.strftime("%Y-%m-%d %H:%M:%S")

          #title
          title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#main > div > div > div.col > div:nth-child(1) > div > h1 > span"))).text
          
          #author
          try : 
            author =  wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='main']/div/div/div[1]/div[1]/div/div/div[1]/div[1]/div/span/a"))).text
          
          except:
            author = " "
        
          #content
          content = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME,"p")))

          all_content = []

          for j in content:
            all_content.append(j.text)
          
          if file.exists() and file2.exists():
              existed_news = pd.read_csv(csv_file_checking)

              record = existed_news[existed_news["Subcategory"] == field_category]
              if len(record.index) != 0 :
                existed_date = record['Time Created'].iloc[0]
                existed_title = record['Title'].iloc[0]

                date_time = datetime.strptime(time_created, "%Y-%m-%d %H:%M:%S")
                exist_date = datetime.strptime(existed_date, "%Y-%m-%d %H:%M:%S")

                if (date_time >= exist_date) and (title != existed_title):
                  valid = True
                  article_details = {'Category': category, 'Subcategory': field_category, 'Time Created': time_created, 'Title' : title, 'Author' : author, 'Content' : all_content}
                  article_list.append(article_details)

                  if i == (len(listOflinks)-1):
                    condition = True

                  else:
                    condition = False                  

                else:
                  valid = False
                  condition = False 
                
              else:
                article_details = {'Category': category, 'Subcategory': field_category, 'Time Created': time_created, 'Title' : title, 'Author' : author, 'Content' : all_content}
                article_list.append(article_details)    
                condition = True                          

          else:
            article_details = {'Category': category, 'Subcategory': field_category, 'Time Created': time_created, 'Title' : title, 'Author' : author, 'Content' : all_content}
            article_list.append(article_details)
            condition = True

          i += 1

        #check if need to go to the next page
        if condition:
          driver.get(c)
          count += 1
          a = 0
          while a < count: 
            try:
              #go to next page
              driver.find_element_by_link_text("Next »").click()
              nextPage = True
            
            except:
              nextPage = False
            a += 1

        else:
          nextPage = False

      if article_list != []:
        results = pd.DataFrame(article_list)
        results["Time Created"] = pd.to_datetime(results["Time Created"], format="%Y-%m-%d %H:%M:%S")
        results = results.sort_values(by="Time Created")
            
        if file.exists ():
            results.to_csv(csv_file, mode = 'a', index=False, header = False)
        else:
            results.to_csv(csv_file, index=False)
        
        last_row = pd.DataFrame(results.tail(1))
        if file2.exists ():
          record = existed_news[existed_news["Subcategory"] == field_category]
          if len(record) != 0:
            last_record_datetime = last_row['Time Created'].iloc[0]

            existed_news.loc[existed_news["Subcategory"] == field_category, 'Time Created'] = last_record_datetime
            existed_news.loc[existed_news["Subcategory"] == field_category, 'Title'] = last_row['Title'].iloc[0]
            existed_news.to_csv(csv_file_checking, index=False)

          else:
            last_row.to_csv(csv_file_checking, mode = 'a', index=False, header = False)

        else:
          last_row.to_csv(csv_file_checking, index=False)
  

csv_file = 'drive/My Drive/fyp/seleniumProject/news.csv' 
crawl_content('https://www.nst.com.my/business', 'BUSINESS')
crawl_content('https://www.nst.com.my/lifestyle', 'LIFESTYLE')
crawl_content('https://www.nst.com.my/sports', 'SPORTS')
crawl_content('https://www.nst.com.my/world', 'WORLD')
crawl_content('https://www.nst.com.my/opinion', 'OPINION')

final_results = pd.read_csv('drive/My Drive/fyp/seleniumProject/news.csv')
final_results["Time Created"] = pd.to_datetime(final_results["Time Created"], format="%Y-%m-%d %H:%M:%S")
final_results = final_results.sort_values(by="Time Created")
final_results = final_results.drop_duplicates()
final_results.to_csv(csv_file, index=False)